In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset
import time
import pandas as pd


In [4]:
X_train = pd.read_csv('/home/niiquaye/ECE457B/data/FashionMNISTwithaTwistW25/x_train.csv') # every row is an image vector
y_train = pd.read_csv('/home/niiquaye/ECE457B/data/FashionMNISTwithaTwistW25/y_train.csv')
X_test = pd.read_csv('/home/niiquaye/ECE457B/data/FashionMNISTwithaTwistW25/x_test.csv')
y_test = pd.read_csv('/home/niiquaye/ECE457B/data/FashionMNISTwithaTwistW25/y_test.csv')


In [5]:
class CNNClassifier(nn.Module):
    def __init__(self, num_classes=5):
        super(CNNClassifier, self).__init__()
        
        # Convolutional Layer 1
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)  # 3x3 filter
        
        # Convolutional Layer 2
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)  # 3x3 filter
        
        # Max Pooling
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        
        # Fully Connected Layer
        self.fc1 = nn.Linear(32 * 196, 512)  # Adjusted for the flattened size after pooling
        
        # Output Layer (Softmax will be applied outside the model during loss computation)
        self.fc2 = nn.Linear(512, num_classes)
        
        # Activation function
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))  # First convolutional layer
        x = self.pool(x)  # Max pooling
        x = self.relu(self.conv2(x))  # Second convolutional layer
        x = self.pool(x)  # Max pooling again
        
        x = x.view(-1, 32 * 196)  # Flatten the output for the fully connected layer
        x = self.relu(self.fc1(x))  # Fully connected layer 1
        x = self.fc2(x)  # Output layer
        
        return x

In [6]:
X_train_tensor = torch.tensor(X_train.to_numpy(), dtype=torch.float32).unsqueeze(1)  # Convert X_train to tensor
y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.long).squeeze()  # Convert y_train to tensor (for classification tasks)

# Create a TensorDataset
train_data = TensorDataset(X_train_tensor, y_train_tensor)

# Create DataLoader
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)


In [21]:
X_test_tensor = torch.tensor(X_test.to_numpy(), dtype=torch.float32).unsqueeze(1)  # Convert X_test to tensor
y_test_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.long).squeeze()  # Convert y_test to tensor (for classification tasks)

# Create a TensorDataset
test_data = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoader
test_loader = DataLoader(test_data, batch_size=32, shuffle=True)


In [14]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'

In [15]:
# Model, optimizer, loss function
model = CNNClassifier(num_classes=5).to(device)  # Move model to GPU
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [19]:
num_epochs = 1000

In [17]:
print(torch.min(y_train_tensor), torch.max(y_train_tensor))


tensor(0) tensor(4)


In [18]:
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    for inputs, labels in train_loader:
        # Move input and labels to GPU
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute the loss
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

Epoch 1/10000, Loss: 0.3443088979856111
Epoch 2/10000, Loss: 0.06857803960667613
Epoch 3/10000, Loss: 0.056234381636049754
Epoch 4/10000, Loss: 0.04815908947528915
Epoch 5/10000, Loss: 0.04372754931092689
Epoch 6/10000, Loss: 0.03196353463035324
Epoch 7/10000, Loss: 0.030503435509166715
Epoch 8/10000, Loss: 0.02384381555101644
Epoch 9/10000, Loss: 0.019581133700007285
Epoch 10/10000, Loss: 0.020745051013027486
Epoch 11/10000, Loss: 0.01720188322557773
Epoch 12/10000, Loss: 0.013436638179338793
Epoch 13/10000, Loss: 0.014987804916504872
Epoch 14/10000, Loss: 0.011302857817613171
Epoch 15/10000, Loss: 0.01381885617048043
Epoch 16/10000, Loss: 0.011711870369771733
Epoch 17/10000, Loss: 0.006439810393714145
Epoch 18/10000, Loss: 0.012473248863797094
Epoch 19/10000, Loss: 0.009926551985522976
Epoch 20/10000, Loss: 0.012829931373305218
Epoch 21/10000, Loss: 0.005583854355518864
Epoch 22/10000, Loss: 0.00866984010847885
Epoch 23/10000, Loss: 0.012955152491681077
Epoch 24/10000, Loss: 0.006790

KeyboardInterrupt: 

In [12]:
torch.cuda.is_available()


True

In [22]:
# After training, you can test your model
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient calculation during testing
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Accuracy: {100 * correct / total}%")

Accuracy: 98.46%
